## Install libraries relevant to the task

In [2]:
!pip install BeautifulSoup4
!pip install lxml
!pip install requests
!pip install geopy
import  requests
import json

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

### Scraping the Data
1. Gets the wikipedia page as variable source.
2. Create soup object which parses the source variable.
3. Create empty variables data and columns to simplify data frame creation later.
4. Create a nested for loop that goes through:</br>
    A. Each individual row ('tr').</br>
    B. To append each header ('th')  and standard cell ('td') to list variable section.</br>
        a. Uses nested for loop index 0 of section to determine headers and assign it to list variable column.
        b. Else assigns all other rows to list variable data. 
5. Create Pandas data frame canada_df from list variables data and columns.
6. Use head function to print the first 15 rows of our new data frame.

In [44]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(source.text, 'lxml')
data=[]
columns=[]
table=soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section=[]
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    if (index==0):
        columns = section
    else:
        data.append(section)

canada_df=pd.DataFrame(data=data, columns=columns)
canada_df.head(15)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"


#### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [51]:
canada_df = canada_df[canada_df['Borough'] != 'Not assigned']
canada_df.head(20)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


#### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [63]:
canada_df = canada_df.groupby(['Postal Code', 'Borough'])['Neighborhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()
for index, row in canada_df.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']
canada_df.head(20)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


#### Prove code worked

In [66]:
canada_df.loc[canada_df['Postal Code'] == 'M5A']

,Postal Code,Borough,Neighborhood
53,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### In the last cell of your notebook, use the .shape method to print the number of rows of your data frame.

In [70]:
canada_df.shape

(103, 3)